In [18]:
using DelimitedFiles, Dierckx

In [16]:
FILE_F_MAP = "../data/F_REFERENCE.txt"
FILE_PS = "../data/WideA_ZA_pk.dat"
FILE_BACKGROUND = "../data/WideA_ZA_background.dat";

In [17]:
struct BackgroundData
     z::AbstractVector{T} where {T}
     #conftime::AbstractVector{T} where {T}
     comdist::AbstractVector{T} where {T}
     #angdist::AbstractVector{T} where {T}
     #lumdist::AbstractVector{T} where {T}
     D::AbstractVector{T} where {T}
     f::AbstractVector{T} where {T}
     ℋ::AbstractVector{T} where {T}
     #ℋ_p::AbstractVector{T} where {T}

     function BackgroundData(file::String, z_min = 0.05, z_max = 0.2;
          names = NAMES_BACKGROUND, h = h_0)

          data = readdlm(file, comments = true)
          N_z_MAX = findfirst(z -> z <= z_max, data[:, 1]) - 1
          N_z_MIN = findfirst(z -> z <= z_min, data[:, 1]) + 1

          data_dict = Dict([name => reverse(data[:, i][N_z_MAX:N_z_MIN])
                            for (i, name) in enumerate(names)]...)

          com_H = data_dict["H [1/Mpc]"] ./ h ./ (1.0 .+ data_dict["z"])
          #conf_time = data_dict["conf. time [Mpc]"] .* h
          #spline_com_H = Spline1D(reverse(conf_time), reverse(com_H))
          #com_H_p = [Dierckx.derivative(spline_com_H, t) for t in conf_time]

          new(
               data_dict["z"],
               #conf_time,
               data_dict["comov. dist."] .* h,
               #data_dict["ang.diam.dist."] .* h,
               #data_dict["lum. dist."] .* h,
               data_dict["gr.fac. D"],
               data_dict["gr.fac. f"],
               com_H,
               #com_H_p,
          )
     end
end

struct BackgroundSplines
     z_of_s
     s_of_z
     D_of_s
     f_of_s
     ℋ_of_s
     ℛ_of_s

     function BackgroundSplines(BD::BackgroundData)
          ℛs = 1.0 .- 1.0 ./ (BD.ℋ .* BD.comdist)
          new(
               Spline1D(BD.comdist, BD.z),
               Spline1D(BD.z, BD.comdist),
               Spline1D(BD.comdist, BD.D),
               Spline1D(BD.comdist, BD.f),
               Spline1D(BD.comdist, BD.ℋ),
               Spline1D(BD.comdist, ℛs)
          )
     end
end

struct Point
     z::Float64
     #conftime::Float64
     comdist::Float64
     #angdist::Float64
     #lumdist::Float64
     D::Float64
     f::Float64
     ℋ::Float64
     #ℋ_p::Float64
     ℛ::Float64

     Point(z, comdist, D, f, ℋ, ℛ) = new(z, comdist, D, f, ℋ, ℛ)
     function Point(s, BS::BackgroundSplines)
          new(BS.z_of_s(s), s, BS.D_of_s(s), BS.f_of_s(s),
               BS.ℋ_of_s(s), BS.ℛ_of_s(s))
     end
end

In [19]:
function ξ_doppler(P1::Point, P2::Point, y; enhancer = 1.0)
     s1, D1, f1, ℋ1, ℛ1 = P1.comdist, P1.D, P1.f, P1.ℋ, P1.ℛ
     s2, D2, f2, ℋ2, ℛ2 = P2.comdist, P2.D, P2.f, P2.ℋ, P2.ℛ

     delta_s = s(P1.comdist, P2.comdist, y)
     prefac = D1 * D2 * f1 * f2 * ℛ1 * ℛ2 * ℋ1 * ℋ2
     c1 = 3 * s1 * s2 - 2 * y * (s1^2 + s2^2) + s1 * s2 * y^2

     parenth = I00(delta_s) / 45.0 + I20(delta_s) / 31.5 + I40(delta_s) / 105.0

     first = prefac * (c1 * parenth + I02(delta_s) * y * delta_s^2 / 3.0)

     return enhancer * first
end

ξ_doppler (generic function with 1 method)

In [ ]:
function integrand_on_mu_doppler(P1::Point, P2::Point, μ; 
     L::Integer = 0, enhancer = 1.0, use_windows::Bool = true)
     if use_windows == true
          ϕ_s2 = ϕ(s2(s1, s, μ))
          (ϕ_s2 > 0.0) || (return 0.0)
          val = ξ_doppler(P1, P2, y(s1, s, μ), enhancer = enhancer)
          return val * ϕ_s2 * spline_F(s / s1, μ) * Pl(μ, L)
     else
          val = ξ_doppler(P1, P2, y(s1, s, μ), enhancer = enhancer)
          return val * Pl(μ, L)
     end
end

In [21]:
using GridInterpolations

In [29]:
struct new_WindowF
     xs::AbstractVector{T1} where {T1}
     μs::AbstractVector{T2} where {T2}
     Fs::AbstractArray{T3,2} where {T3}
    spline

     function new_WindowF(file::String)
          data = readdlm(file, comments = true)
          xs, μs, Fs = data[:, 1], data[:, 2], data[:, 3]
          @assert size(xs) == size(μs) == size(Fs) "xs, μs and Fs must have the same length!"

          new_xs = unique(xs)
          new_μs = unique(μs)
          new_Fs =
               if xs[2] == xs[1] && μs[2] ≠ μs[1]
                    transpose(reshape(Fs, (length(new_μs), length(new_xs))))
               elseif xs[2] ≠ xs[1] && μs[2] == μs[1]
                    reshape(Fs, (length(new_xs), length(new_μs)))
               else
                    throw(ErrorException("What kind of convenction for the file $file" *
                                         " are you using? I do not recognise it."))
               end
        grid = GridInterpolations.RectangleGrid(new_μs, new_xs)
        spline(x, μ) = GridInterpolations.interpolate(grid, new_Fs, [x, μ])
          new(new_xs, new_μs, new_Fs, spline)
     end
end

In [30]:
str = new_WindowF("F_REFERENCE.txt")

new_WindowF([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9  …  2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0], [-1.0, -0.99, -0.98, -0.97, -0.96, -0.95, -0.94, -0.93, -0.92, -0.91  …  0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0], [0.0 20.315581703061408 … 20.315581372967298 0.0; 0.0 20.121689187178195 … 19.984420159530657 0.0; … ; 0.0 1.7647802990836603 … 18.563437720176587 0.0; 0.0 1.7647802990836603 … 18.563437720176587 0.0], var"#spline#18"{RectangleGrid{2}, LinearAlgebra.Transpose{Float64, Matrix{Float64}}}(RectangleGrid{2}([-1.0, -0.99, -0.98, -0.97, -0.96, -0.95, -0.94, -0.93, -0.92, -0.91  …  0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0],[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9  …  2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0],), [0.0 20.315581703061408 … 20.315581372967298 0.0; 0.0 20.121689187178195 … 19.984420159530657 0.0; … ; 0.0 1.7647802990836603 … 18.563437720176587 0.0; 0.0 1.7647802990836603 … 18.563437720176587 0.0]))

In [ ]:
plot(str.xs, str.μs, str.spline(a) ; 
    xl="x", yl="y", zl="z", st = :surface, 
    c=cgrad([:yellow, :red]), size=(300, 300))